In [2]:
import pandas as pd

# Задание 1

Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

если источник traffic_source равен yandex или google, то в source_type ставится organic
для источников paid и email из России - ставим ad
для источников paid и email не из России - ставим other
все остальные варианты берем из traffic_source без изменений


In [86]:
log = pd.read_csv('../data2/visit_log.csv', sep=';')
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [104]:
log.loc[lambda row: row['traffic_source'].str.contains('yandex|google'), 'source_type'] = 'organic'
log.loc[lambda row: (row['traffic_source'].str.contains('paid|email')) & (row.region == 'Russia'), 'source_type'] = 'ad'
log.loc[lambda row: (row['traffic_source'].str.contains('paid|email')) & (row.region != 'Russia'), 'source_type'] = 'other'
log['source_type'] = log.source_type.fillna(log['traffic_source'])
log

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic
...,...,...,...,...,...,...,...
18933,1550094288,57e5ba8560,https://host.ru/c2382eb3d6afc8d0f3,Belarus,98b19810d0,paid,other
18934,1550094296,6f9389ec1b,https://host.ru/f1eb4601740d627ab0,Russia,32ebb20c13,paid,ad
18935,1550094308,e8cf2eb8e6,https://host.ru/a5dda93e70318570c0,Belarus,b85baa8c73,yandex,organic
18936,1550094314,79530b9a67,https://host.ru/6fda01ec57f23abc9e,Russia,e154b06121,paid,ad


# Задание 2

В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

Прочитайте содержимое файла с датафрейм
Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [123]:
df = pd.read_csv('../data2/URLs.txt')
df.loc[df.url.str.contains('/\d{8}-')]

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...
...,...
88,/cis/35984145-kreml-prokommentiroval-soobschen...
89,/video/36071019-olimpiyskie-obekty-rio-prevrat...
90,/science/36151301-nazvano-posledstvie-zloupotr...
91,/incidents/36027330-vospitatelnitsu-zatravili-...


# Задание 3

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [174]:
rt = pd.read_csv('../data2/ml-latest-small/ratings.csv')
rt

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [187]:
filtered = rt.groupby('userId').filter(lambda x: x['rating'].count() > 100).groupby('userId')['timestamp'].agg(['min', 'max'])
filtered['min'] = pd.to_datetime(filtered["min"], unit="s")
filtered['max'] = pd.to_datetime(filtered["max"], unit="s")
filtered['ttl'] = filtered['max'] - filtered['min']
filtered['ttl'].mean()

# pd.to_datetime(filtered.ttl.agg('mean'), unit='s', origin='unix')

Timedelta('463 days 21:28:27.449612408')

# Задание 4

Дана статистика услуг перевозок клиентов компании по типам (см. файл “Python_13_join.ipynb” в разделе Материалы для лекции «Продвинутый pandas» ---- Ноутбуки к лекции «Продвинутый pandas»).
Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента

In [188]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1',
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [195]:
tab1 = pd.concat([air, auto, rzd])
tab1.pivot_table(index='client_id')

,air_revenue,auto_revenue,rzd_revenue
client_id,,,
111,NaN,NaN,1093.0
112,NaN,NaN,2810.0
113,NaN,57483.0,10283.0
114,NaN,83.0,5774.0
115,81.0,912.0,981.0
116,4.0,4834.0,NaN
117,13.0,98.0,NaN
118,173.0,NaN,NaN


In [199]:
tab1.merge(client_base, on="client_id", how='left')

,client_id,air_revenue,auto_revenue,rzd_revenue,address
0,115,81.0,NaN,NaN,ЗЖБИиДК 1
1,116,4.0,NaN,NaN,Строителей 18
2,117,13.0,NaN,NaN,Панфиловская 33
3,118,173.0,NaN,NaN,Мастеркова 4
4,113,NaN,57483.0,NaN,Левобережная 1а
5,114,NaN,83.0,NaN,Мира 14
6,115,NaN,912.0,NaN,ЗЖБИиДК 1
7,116,NaN,4834.0,NaN,Строителей 18
8,117,NaN,98.0,NaN,Панфиловская 33
9,111,NaN,NaN,1093.0,Комсомольская 4
